In [1]:
!pip install -q evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 40.1 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.25.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.1.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12==25.6.*,

In [2]:
!pip install -q bitsandbytes==0.43.0
!pip install -q transformers==4.44.2 accelerate==0.34.2 peft==0.12.0 evaluate
!pip install -q datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 17.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.2 MB/s eta 0:00:00:00:0100:01
ERROR: pip's 

In [3]:
# =========================================================
# ✅ 4-bit Fine-tuning of T5-small (CNN/DailyMail) for Kaggle
# =========================================================

import torch
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM,
    BitsAndBytesConfig, Seq2SeqTrainingArguments,
    Seq2SeqTrainer, DataCollatorForSeq2Seq
)
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType
import evaluate
import numpy as np

# =========================================================
# Load small dataset subset for Kaggle memory
# =========================================================
dataset = load_dataset("/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail")

small_train = dataset["train"].select(range(2000))
small_val = dataset["validation"].select(range(200))

# =========================================================
# Tokenizer and 4-bit model
# =========================================================
model_name = "t5-small"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

# =========================================================
# Add LoRA adapter (makes quantized model trainable)
# =========================================================
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

model = get_peft_model(model, lora_config)

# =========================================================
# Preprocess
# =========================================================
max_input_len = 512
max_target_len = 128

def preprocess_fn(batch):
    inputs = ["summarize: " + doc for doc in batch["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_len, truncation=True)
    labels = tokenizer(batch["highlights"], max_length=max_target_len, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = small_train.map(preprocess_fn, batched=True, remove_columns=small_train.column_names)
tokenized_val = small_val.map(preprocess_fn, batched=True, remove_columns=small_val.column_names)

# =========================================================
# Data collator and metrics
# =========================================================
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
rouge = evaluate.load("rouge")

def postprocess_text(preds, labels):
    preds = [p.strip() for p in preds]
    labels = [l.strip() for l in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple): preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    # ✅ Compatible with new ROUGE format
    if isinstance(list(result.values())[0], dict):
        result = {k: round(v["fmeasure"] * 100, 2) for k, v in result.items()}
    else:
        result = {k: round(v * 100, 2) for k, v in result.items()}
    return result

# =========================================================
# Training setup
# =========================================================
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5_small_cnn_4bit",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,          # ✅ Lower epochs for Kaggle speed
    learning_rate=2e-4,
    fp16=True,
    predict_with_generate=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    logging_steps=50,
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# =========================================================
# Train and save model
# =========================================================
trainer.train()
trainer.save_model("./t5_small_cnn_4bit")


2025-11-02 15:40:16.561190: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762098016.785670      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762098016.846406      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

False

===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/lib/python3.11/dist-packages/cv2/../../lib64'), PosixPath('/usr/local/nvidia/lib')}
The following directories listed in your path were found to be non-existent: {PosixPath('https'), PosixPath('//www.kaggle.com')}
The following directories listed in your path were found to be non-existent: {PosixPath('gcr.io/kaggle-gpu-images/python@sha256'), PosixPath('23c0e8309600afc19063b29d98f0f1e58dc32d03369b15ad204d43d90add6325')}
The following directories listed in your path were found to be non-existent: {PosixPath('https'), PosixPath('//colab\\.(sandbox|research)\\.google\\.com')}
The following directories listed in your path were found to be non-existent: {PosixPath('/kaggle/lib/kagglegym')}
The following directories listed in your path were found to be non-existent: {PosixPath('https'), PosixPath('//dp.kaggl

/usr/local/lib/python3.11/dist-packages/bitsandbytes/cuda_setup/main.py:183: UserWarning: Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes


  warn(msg)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/cuda_setup/main.py:183: UserWarning: /usr/local/lib/python3.11/dist-packages/cv2/../../lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0', 'libcudart.so.12.1', 'libcudart.so.12.2'] as expected! Searching further paths...
  warn(msg)


RuntimeError: Failed to import transformers.integrations.bitsandbytes because of the following error (look up to see its traceback):

        CUDA Setup failed despite GPU being available. Please run the following command to get more information:

        python -m bitsandbytes

        Inspect the output of the command and see if you can locate CUDA libraries. You might need to add them
        to your LD_LIBRARY_PATH. If you suspect a bug, please take the information from python -m bitsandbytes
        and open an issue at: https://github.com/TimDettmers/bitsandbytes/issues

In [ ]:
# =========================================================
# 🚀 Gradio App for T5-small Summarization (4-bit LoRA)
# =========================================================

import torch
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig

# =========================================================
# Load fine-tuned model (4-bit quantized)
# =========================================================
model_dir = "./t5_small_cnn_4bit"   # Path to your fine-tuned model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# ✅ Load tokenizer & model
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_dir,
    quantization_config=bnb_config,
    device_map="auto"
)
model.eval()  # Put model in eval mode for inference

# =========================================================
# Summarization function
# =========================================================
def summarize_text(article):
    article = article.strip()
    if not article:
        return "⚠️ Please enter a valid article."

    # Prepare input
    input_text = "summarize: " + article
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(model.device)

    # Generate summary
    with torch.inference_mode():
        summary_ids = model.generate(
            **inputs,
            max_length=150,
            min_length=30,
            num_beams=4,
            length_penalty=2.0,
            temperature=0.7,
            early_stopping=True
        )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary.strip()

# =========================================================
# Create Gradio UI
# =========================================================
demo = gr.Interface(
    fn=summarize_text,
    inputs=gr.Textbox(
        lines=10,
        label="📰 Enter News Article",
        placeholder="Paste or type a news article here..."
    ),
    outputs=gr.Textbox(
        lines=5,
        label="🧠 Generated Summary"
    ),
    title="T5-small Summarizer (4-bit LoRA)",
    description="Fine-tuned on CNN/DailyMail using 4-bit NF4 quantization and LoRA adapters. Runs efficiently on Kaggle GPU!"
)

# =========================================================
# Launch Gradio app
# =========================================================
demo.launch(share=True, debug=True)
